# Bayesian linear mixed model for two repeated measures group

Dataset used in this analysis os taken from https://www.sheffield.ac.uk/mash/statistics/datasets "cholesterol"

In [71]:
# Import all the relevant python packages for the analysis below
import os
import numpy as np
import pandas as pd
import pystan as ps
import seaborn as sns


In [131]:
# Import data into the notebook from the file path to the repository on the local system
os.chdir(r"C:\Users\harri\OneDrive\Documents\Repositories\Statistical-methods-for-research-workers-bayes-for-psychologists-and-neuroscientists\Data")

# Generating the pandas dataset
df = pd.read_csv("Cholesterol_R.csv")

# Using this example first to demonstrate the simplest case with two groups
# as such recducing the df by dropping the After 8 weeks,
df_reduced = df.drop(columns=["After8weeks", "Margarine"])


# Convert the dataframe from wide to long format.
df_longForm = pd.melt(df_reduced, id_vars=['ID'], value_vars=['Before','After4weeks'],
                      var_name='Predictors', value_name='DV')


# Adding column for the two grups with effect coding values for input in design matrix.
df_longForm["Predictors.cont"] = np.where(df_longForm['Predictors'] == "Before", '0.5', '-0.5')
df_longForm["Predictors.cont"] = df_longForm["Predictors.cont"].astype("float64")
df_longForm

,ID,Predictors,DV,Predictors.cont
0,1,Before,6.42,0.5
1,2,Before,6.76,0.5
2,3,Before,6.56,0.5
3,4,Before,4.80,0.5
4,5,Before,8.43,0.5
5,6,Before,7.49,0.5
6,7,Before,8.05,0.5
7,8,Before,5.05,0.5
8,9,Before,5.77,0.5
9,10,Before,3.91,0.5


In [161]:
Two_group_LMM_VaryingIntercept = """
data{
int<lower=0> N; //number of data points
real y[N]; //Dependent variable

real<lower=-0.5, upper=0.5> x[N]; //predictor
int<lower=1> J; //number of partipants

int<lower=1, upper=J> ID[N]; // Partipants ID

}


parameters{
vector[2] beta; //Beta coefficients
vector[J] u; //subject intercepts
real<lower=0> sigma_e; //error sd
real<lower=0> sigma_u; //subj sd


}

model{
real mu;
//priors
 u ~ normal(0, sigma_u); //subj random effects
 
// likelihood
for (i in 1:N){

//beat[1] intercept term
mu = beta[1] + u[ID[i]] +  x[i] * beta[2] ;

y[i] ~ normal(mu, sigma_e);

 }
}
"""

In [162]:
sm = ps.StanModel(model_code = Two_group_LMM_VaryingIntercept)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_2b55c620f2810f5ecb2b520558e79e44 NOW.


In [163]:
Number_of_categories = pd.Categorical(df_longForm.ID)

Number_of_categories[-1]

18

In [164]:
data = {'ID': df_longForm["ID"].values,
       'y': df_longForm["DV"].values,
       'N': len(df_longForm),
       'J': Number_of_categories[-1],
       'x': df_longForm["Predictors.cont"].values}


In [169]:
fit = sm.sampling(data = data , iter=10000, chains=4, seed= 302675,warmup=1000)


In [170]:
print(fit)

Inference for Stan model: anon_model_2b55c620f2810f5ecb2b520558e79e44.
4 chains, each with iter=10000; warmup=1000; thin=1; 
post-warmup draws per chain=9000, total post-warmup draws=36000.

          mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
beta[1]   6.13  6.8e-3    0.3   5.54   5.95   6.13   6.32   6.74   1930    1.0
beta[2]   0.57  4.3e-4   0.04   0.49   0.54   0.57   0.59   0.65   9162    1.0
u[1]   -5.8e-3  6.8e-3   0.31  -0.64   -0.2-5.6e-3   0.19   0.62   2056    1.0
u[2]      0.35  6.8e-3   0.31  -0.28   0.15   0.35   0.54   0.97   2052    1.0
u[3]      0.06  6.8e-3   0.31  -0.56  -0.13   0.06   0.26   0.68   2071    1.0
u[4]     -1.59  6.8e-3   0.31  -2.22  -1.78  -1.59  -1.39  -0.97   2074    1.0
u[5]      1.93  6.8e-3   0.31   1.31   1.73   1.93   2.12   2.55   2064    1.0
u[6]      1.17  6.8e-3   0.31   0.54   0.97   1.17   1.36    1.8   2070    1.0
u[7]      1.51  6.8e-3   0.31   0.88   1.32   1.51   1.71   2.13   2067    1.0
u[8]     -1.28  6.8